In [1]:
%load_ext autoreload
%autoreload
import pandas as pd
import datetime 
import pickle
import sys
import os
sys.path.append("/homes/gws/kdeeds/FrequencyBounds/BayesCard")
sys.path.append("/homes/gws/kdeeds/FrequencyBounds/Source")
from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.Bayescard_BN import Bayescard_BN, build_meta_info
from DeepDBUtils.evaluation.utils import timestamp_transorform

In [40]:
tableNames = ['badges', 'comments', 'postHistory', 'postLinks', 'posts', 'tags', 'users', 'votes']
for tableName in tableNames:
    tableCSVLocation = "/homes/gws/kdeeds/FrequencyBounds/Data/Stats/stats_simplified/" + tableName + ".csv"
    tableOutputLocation = "/homes/gws/kdeeds/FrequencyBounds/Data/Stats/stats_simplified_bayes_card/" + tableName + ".csv"
    table = pd.read_csv(tableCSVLocation)
    for column in table.columns:
        if not pd.api.types.is_numeric_dtype(table[column]):
            table[column] = table[column].apply(lambda x: timestamp_transorform("'" + x + "'"))          
    table.to_csv(tableOutputLocation, header=False, index=False)

In [34]:
table = pd.read_csv("/homes/gws/kdeeds/FrequencyBounds/Data/Stats/stats_simplified_bayes_card/" + 'posts' + ".csv")
table["1279566732000000000"]

0        1279566777000000000
1        1279566808000000000
2        1279566811000000000
3        1279566883000000000
4        1279566884000000000
                ...         
91970    1410651939000000000
91971    1410651965000000000
91972    1410658074000000000
91973    1410660208000000000
91974    1410660563000000000
Name: 1279566732000000000, Length: 91975, dtype: int64

In [20]:
hdf_path = "/homes/gws/kdeeds/FrequencyBounds/Data/JOB/JOB_hdf"
meta_data_path = hdf_path + '/meta_data.pkl'
from Schemas.imdb.schema import gen_job_light_imdb_schema
schema = gen_job_light_imdb_schema("/homes/gws/kdeeds/FrequencyBounds/Data/JOB")
prep = JoinDataPreparator(meta_data_path, schema, max_table_data=20000000)
print(f"BN will be trained on the full outer join of following relations")
for relationship_obj in schema.relationships:
    print(relationship_obj.identifier)

BN will be trained on the full outer join of following relations
movie_info_idx.movie_id = title.id
movie_info.movie_id = title.id
cast_info.movie_id = title.id
movie_keyword.movie_id = title.id
movie_companies.movie_id = title.id


In [21]:
algorithm = "chow-liu"
max_parents = 1
sample_size = 200000
model_folder = "/homes/gws/kdeeds/FrequencyBounds/StatObjects/BayesCardEnsembles/JOBLight"
size = 0
memorySize = 0
for i, relationship_obj in enumerate(schema.relationships):
    df_sample_size = 10000000
    relation = [relationship_obj.identifier]
    print("training on relation", relation)
    df, meta_types, null_values, full_join_est = prep.generate_n_samples(
        df_sample_size, relationship_list=relation, post_sampling_factor=10)
    print(df.columns)
    columns = list(df.columns)
    assert len(columns) == len(meta_types) == len(null_values)
    
    meta_info = build_meta_info(df.columns, null_values)
    bn = Bayescard_BN(schema, relation, column_names=columns, full_join_size=len(df),
                      table_meta_data=prep.table_meta_data, meta_types=meta_types, null_values=null_values,
                      meta_info=meta_info)
    model_path = model_folder + f"/{i}_{algorithm}_{max_parents}.pkl"
    bn.build_from_data(df, algorithm=algorithm, max_parents=max_parents, ignore_cols=['Id'],
                       sample_size=sample_size)
    pickle.dump(bn, open(model_path, 'wb'), pickle.HIGHEST_PROTOCOL)
    size += os.path.getsize(model_path)
    memorySize += sys.getsizeof(bn)
    print(f"model saved at {model_path}")
print(size)

training on relation ['movie_info_idx.movie_id = title.id']
Index(['title.kind_id', 'title.production_year',
       'title.mul_movie_info_idx.movie_id_nn', 'title.mul_movie_info.movie_id',
       'title.mul_cast_info.movie_id', 'title.mul_movie_keyword.movie_id',
       'title.mul_movie_companies.movie_id',
       'movie_info_idx.movie_info_idx_nn', 'movie_info_idx.info_type_id'],
      dtype='object')
Discretizing table takes 21.3177490234375 secs
Structure learning took 5.286877632141113 secs.
done, parameter learning took 4.485618591308594 secs.
model saved at /homes/gws/kdeeds/FrequencyBounds/StatObjects/BayesCardEnsembles/JOBLight/0_chow-liu_1.pkl
training on relation ['movie_info.movie_id = title.id']
Index(['movie_info.movie_info_nn', 'movie_info.info_type_id', 'title.kind_id',
       'title.production_year', 'title.mul_movie_info_idx.movie_id',
       'title.mul_movie_info.movie_id_nn', 'title.mul_cast_info.movie_id',
       'title.mul_movie_keyword.movie_id',
       'title.mul

In [22]:
print(size)
print(memorySize)

1625051
240


In [76]:
os.path.getsize("/homes/gws/kdeeds/FrequencyBounds/StatObjects/SafeBound_4_Stats.pkl")

3584898

In [75]:
from SafeBoundUtils import *
stats = pickle.load(open("/homes/gws/kdeeds/FrequencyBounds/StatObjects/SafeBound_4_Stats.pkl", 'rb'))

In [77]:
stats.memory()

Table: USERS
Filter Col: USERS.VIEWS
Interval Footprint: 10884
Range Footprint: 1104.0
Equality Bloom Filter Footprint: 1334.0
Equality Outlier Footprint: 768.0
Equality Max Footprint: 24.0
TriGram Footprint: 0
Null Footprint: 48.0
Filter Col: USERS.UPVOTES
Interval Footprint: 10868
Range Footprint: 1104.0
Equality Bloom Filter Footprint: 1268.0
Equality Outlier Footprint: 768.0
Equality Max Footprint: 24.0
TriGram Footprint: 0
Null Footprint: 48.0
Filter Col: USERS.REPUTATION
Interval Footprint: 10872
Range Footprint: 1104.0
Equality Bloom Filter Footprint: 3076.0
Equality Outlier Footprint: 768.0
Equality Max Footprint: 24.0
TriGram Footprint: 0
Null Footprint: 48.0
Filter Col: USERS.DOWNVOTES
Interval Footprint: 10884
Range Footprint: 1104.0
Equality Bloom Filter Footprint: 400.0
Equality Outlier Footprint: 408.0
Equality Max Footprint: 24.0
TriGram Footprint: 0
Null Footprint: 48.0
Filter Col: USERS.CREATIONDATE_String_USERS.REPUTATION_String
Interval Footprint: 0
Range Footprint: 

2775192.0